# Perla's code for only 26 classes -wonbin

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import os

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"    

In [ ]:
dataset=np.load('wav16.npz')

# Dataset preparation

Splitting the dataset into train & validation set

In [ ]:
split=3
# Split to train and val
train_sounds = []
train_labels = []
val_sounds = []
val_labels = []
for i in range(1, 6):
    sounds = dataset['fold{}'.format(i)].item()['sounds']
    labels = dataset['fold{}'.format(i)].item()['labels']
    if i == split:
        val_sounds.extend(sounds)
        val_labels.extend(labels)
    else:
        train_sounds.extend(sounds)
        train_labels.extend(labels)

# I just added this one cell

In [ ]:
chosen_labels = [0, 5, 10, 14, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28 ,29, 30, 32, 33, 35, 36, 37, 39, 42, 43]
a_labels = []
a_sounds = []
b_labels = []
b_sounds = []
for i in range(0, 1600):
    if(train_labels[i] in chosen_labels):
        new_label = chosen_labels.index(train_labels[i])
        a_labels.insert(-1, new_label)
        a_sounds.insert(-1, train_sounds[i])

for i in range(0, 400):
    if(val_labels[i] in chosen_labels):
        new_label = chosen_labels.index(val_labels[i])
        b_labels.insert(-1, new_label)
        b_sounds.insert(-1, val_sounds[i])
        
train_labels = a_labels
train_sounds = a_sounds
val_labels = b_labels
val_sounds = b_sounds

print(len(train_labels))
print(len(val_labels))

In [ ]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [ ]:
train_labels=one_hot_encode(train_labels)
val_labels=one_hot_encode(val_labels)

Normalize to have value between -1 and 1

In [ ]:
def normalize(sound, factor=32768):
    return [s/factor for s in sound]

Random crop of the sound values to have T-s window of sound

In [ ]:
def random_crop(sound, size=24014):
    cropped_sound=[]
    for s in sound:
        org_size = len(s)
        start = random.randint(0, org_size - size)
        cropped_s=s[start: start + size]
        cropped_sound.append(cropped_s)
    return cropped_sound

In [ ]:
def padding(sound, pad=24014//2):
    if len(sound)<80000:
        padded_sound=[]
        for i in range(len(sound)):
            padded_s=np.pad(sound[i], pad, 'constant')
            padded_sound.append(padded_s)
    return padded_sound

Multi-crop for testing phase

In [ ]:
def multi_crop(sounds,input_length=24014, n_crops=10):
    multi_cropped_sounds=[]
    for s in sounds:
        stride = (len(s) - input_length) // (n_crops - 1)
        multi_cropped_sound = [s[stride * i: stride * i + input_length] for i in range(n_crops)]
        multi_cropped_sounds.append(np.array(multi_cropped_sound))
    return multi_cropped_sounds

In [ ]:
# what is supposed to be the pad parameter ?
#train_sounds=padding(train_sounds, 24014//2) 
#val_sounds=padding(train_sounds, 24014//2)

Size of window = 1.5s (24014)

Normalization constant = 32768 

In [ ]:
train_y=train_labels
train_y.shape

In [ ]:
test_X=multi_crop(normalize(padding(val_sounds)))

In [ ]:
test_X = np.reshape(test_X, (-1, 24014, 1, 1))

In [ ]:
test_X.shape

In [ ]:
test_y=np.repeat(val_labels, 10, axis=0)
test_y.shape

In [ ]:
len(train_sounds)

To improve : do the cross validation

# Model

In [ ]:
import tensorflow as tf

In [ ]:
#n_classes=3
n_classes=26

## Input of the model

Recall that the tensor is 4D : [batch, height, width, channel]

In [ ]:
X = tf.placeholder(tf.float32, shape=[None, 24014,1, 1])
y = tf.placeholder(tf.float32, shape=[None, n_classes])

In [ ]:
keep_prob = tf.placeholder(tf.float32) # 50%

In [ ]:
is_training = tf.placeholder(tf.bool)

## Hyperparameters

In [ ]:
training_epoch = 300 
batch_size = 64
learning_rate = tf.placeholder(tf.float32)
crop_size=10

In [ ]:
def lr(epoch):
    if (0<=epoch<=80):
        return 0.01
    if (80<epoch<=100):
        return 0.001
    if (100<epoch<= 120):
        return 0.0001
    else: return 0.00001

Stride =1

No Padding

In [ ]:
def conv2d(x, W, b,is_training, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID')
    x = tf.nn.bias_add(x, b)
    x = tf.contrib.layers.batch_norm(x, is_training=is_training)
    return tf.nn.relu(x) 

def maxpool2d(x, k_h=2, k_w=2):
    return tf.nn.max_pool(x, ksize=[1, k_h, k_w, 1], strides=[1, k_h, k_w, 1],padding='VALID')

## Parameters of the model

Initialization of the parameters = He initialization

Review the order (does the order actually matters ?)

[heigth_filter, width_filter, depth_filter, number of filters]

In [ ]:
weights = {
    #conv1
    'wc1': tf.get_variable('W0', shape=(8,1,1,40), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
    #conv2
    'wc2': tf.get_variable('W1', shape=(8,1,40,40), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
    #conv3
    'wc3': tf.get_variable('W2', shape=(13,8,1,50), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
    #conv4
    'wc4': tf.get_variable('W3', shape=(5,1,50,50), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
    #fc5
    'wfc5': tf.get_variable('W4', shape=(50*11*14,4096), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
    #fc5
    'wfc6': tf.get_variable('W5', shape=(4096,4096), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
    #output
    'out': tf.get_variable('W6', shape=(4096,n_classes), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)), 
}
biases = {
    'bc1': tf.get_variable('B0', shape=(40), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
    'bc2': tf.get_variable('B1', shape=(40), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
    'bc3': tf.get_variable('B2', shape=(50), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
    'bc4': tf.get_variable('B3', shape=(50), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
    'bfc5': tf.get_variable('B4', shape=(4096), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
    'bfc6': tf.get_variable('B5', shape=(4096), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
    'out': tf.get_variable('B6', shape=(n_classes), initializer=tf.contrib.layers.variance_scaling_initializer(dtype=tf.float32)),
}

## EnvNet

In [ ]:
def env_net(x, weights, biases):  

    conv1 = conv2d(x, weights['wc1'], biases['bc1'], is_training)
    
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'], is_training)
    conv2 = maxpool2d(conv2, k_h=160, k_w=1)
    
    conv2=tf.reshape(conv2, [-1, 150, 40, 1])
    
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'], is_training)
    conv3 = maxpool2d(conv3, k_h=3, k_w=3)
    
    conv4 = conv2d(conv3, weights['wc4'], biases['bc4'], is_training)
    conv4 = maxpool2d(conv4, k_h=3, k_w=1)
    
    # Fully connected layer
    fc5 = tf.reshape(conv4, [-1, weights['wfc5'].get_shape().as_list()[0]])
    fc5 = tf.add(tf.matmul(fc5, weights['wfc5']), biases['bfc5'])
    fc5 = tf.nn.relu(fc5)
    # Drop out
    drop_out_fc5 = tf.nn.dropout(fc5, keep_prob)
    
    fc6 = tf.add(tf.matmul(drop_out_fc5, weights['wfc6']), biases['bfc6'])
    fc6 = tf.nn.relu(fc6)
    drop_out_fc6 = tf.nn.dropout(fc6, keep_prob)
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(drop_out_fc6, weights['out']), biases['out'])
    return out

## Loss & optimizer of the model

In [ ]:
init_momentum = 0.9
momentum = tf.Variable(init_momentum, trainable=False)

In [ ]:
logits = env_net(X, weights, biases)

pred_train=logits

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred_train, labels=y))

pred_test=tf.reduce_sum(tf.nn.softmax(logits),0)

with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9, use_nesterov=True).minimize(cost)

In [ ]:
#Here you check whether the index of the maximum value of the predicted image is equal to the actual labelled image. and both will be a column vector.
#correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

correct_prediction_train = tf.equal(tf.argmax(pred_train, 1), tf.argmax(y, 1))
correct_prediction_test = tf.equal(tf.argmax(pred_test, 0), tf.argmax(y, 1)[0])

#calculate accuracy across all the given images and average them out. 
accuracy_train = tf.reduce_mean(tf.cast(correct_prediction_train, tf.float32))
accuracy_test=tf.cast(correct_prediction_test, tf.float32)

In [ ]:
# Initializing the variables
init = tf.global_variables_initializer()

In [ ]:
with tf.device("/gpu:0"):
    with tf.Session() as sess:
        sess.run(init) 
        train_loss = []
        test_loss = []
        train_accuracy = []
        test_accuracy = []
        summary_writer = tf.summary.FileWriter('./Output', sess.graph)
        for i in range(training_epoch):
            train_X=random_crop(normalize(padding(train_sounds))) # Choosing randomly a 1.5s section
            train_X = np.reshape(train_X, (-1, 24014, 1, 1)) # Reshape to have 1D input
            for batch in range(len(train_X)//batch_size):
                batch_x = train_X[batch*batch_size:min((batch+1)*batch_size,len(train_X))]
                batch_y = train_y[batch*batch_size:min((batch+1)*batch_size,len(train_y))]    
                    # Run optimization op (backprop).
                        # Calculate batch loss and accuracy
                opt = sess.run(optimizer, feed_dict={X: batch_x, y: batch_y, 
                                                         keep_prob:0.5, is_training:True, 
                                                         learning_rate:lr(i)})
                loss, acc = sess.run([cost, accuracy_train], feed_dict={X: batch_x, y: batch_y, 
                                                                         keep_prob:0.5, is_training:True,
                                                                     learning_rate:lr(i)})
            print("Iter " + str(i) + ", Loss= " + \
                              "{:.6f}".format(loss) + ", Training Accuracy= " + \
                              "{:.5f}".format(acc))
            
            acc_test=[]
            for crop in range(len(test_X)//crop_size):
                crop_x = test_X[crop*crop_size:min((crop+1)*crop_size,len(test_X))]
                crop_y = test_y[crop*crop_size:min((crop+1)*crop_size,len(test_y))] 
                
                acc_t= sess.run([accuracy_test], feed_dict={X: crop_x, y : crop_y, 
                                                                           keep_prob:1.0, is_training:False,
                                                            learning_rate:lr(i)})
                acc_test.append(acc_t)
               
            test_acc=np.mean(acc_test)
            #print(len(acc_test))
            
            train_loss.append(loss)
            #test_loss.append(valid_loss)
            train_accuracy.append(acc)
            test_accuracy.append(test_acc)
            print("Testing Accuracy:","{:.5f}".format(test_acc))
        summary_writer.close()

In [ ]:
plt.plot(train_accuracy)
plt.plot(test_accuracy)

TO DO/REMARKS : 
- Accuracy for now : 0.2950 (with my dataset), 0.35750 (with their dataset)
- With only 3 classes (baby crying, glass breaking, coughing) : 0.6250 
- why is training accuracy low ?
- check for silent windows -> remove them
- why 10 crops for testing phase ?
- Accuracy with the new testing scheme + 150 training epochs : 0.365 (with my dataset), 0.48 (with their dataset)
- 230 traning epochs, testing accuracy : 0.54
- Test on another dataset
- Semi-supervised learning ?
- Check to remove silent windows in test set ?